In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Importing Modules

In [ ]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
import re
from keras.applications.vgg16 import VGG16, preprocess_input
from tqdm import tqdm
import cv2
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam

# DataLoader

In [ ]:
# Training set
inp = np.load('/content/drive/MyDrive/Clickbait/MultiModal/Inputs.npy', allow_pickle=True)

# W2V Embedding

In [ ]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(inp[:,1],
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [ ]:
words = set(w2v_model.wv.index_to_key )
X_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in inp[:,1]])

<ipython-input-8-6d8a89087f60>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])


In [ ]:
# Compute sentence vectors by averaging the word vectors for the words contained in the sentence
X_vect_avg = []
for v in X_vect:
    if v.size:
        X_vect_avg.append(v.mean(axis=0))
    else:
        X_vect_avg.append(np.zeros(100, dtype=float))

In [ ]:
# Adding the Input array with vector representations and reshaping them to fit the trained models
for i in range(0, len(X_vect_avg)):
  inp[i][1] = np.reshape(np.array(X_vect_avg[i]), (1,1,100))
  inp[i][0] = np.reshape(inp[i][0], (1, 126, 126))

# Multi-Modal Model

## Weighted Output Function

The accuracy of the models are as follows:

- Text Classifier (Bi-GRU with Word2Vec) = 0.789

- Image Classifier (2-layer CNN) = 0.86


Total Accuracy = Image Accuracy + Text Accuracy

Image Weights = Image_Accuracy/Total Accuracy

Text Weights = Text_Accuracy/Total Accuracy

Total Accuracy = 1(Total Image Accuracy) + 1(Total Text Accuracy) = 2

In [ ]:
def weighted_output(img, text):
  total_acc = float(2)
  img_wgts = float(0.789/2)
  txt_wgts = float(0.86/2)
  op = img_wgts * img + txt_wgts * text
  if op > 0.5:
    return "Clickbait"
  else:
    return "Not Clickbait"

The input to the multi modal model must of the following form:

[Image, Text]

In [ ]:
text_classifier = tf.keras.models.load_model('/content/drive/MyDrive/Clickbait/Text_Dataset/weights/model_BiGRU_w2v') # Text Classifier
image_classifier = tf.keras.models.load_model('/content/drive/MyDrive/Clickbait/Image_Models/CNN') # Image Classifier

In [ ]:
# Setting up the Multimodal Architecture by Combining the different best performing models
def multi_modal_model(X):
  results = []
  for i in X:
    if len(list(X))==1:
      r = text_classifier(i)
      if r > 0.5:
        results.append("Clickbait")
      else:
        results.append("Not Clickbait")
    else:
      results.append(weighted_output(image_classifier.predict(i[0]), text_classifier.predict(i[1])))
  return results

In [ ]:
results = multi_modal_model(inp)

1/1 [==============================] - 0s 18ms/step


In [ ]:
results

['Not Clickbait',
 'Not Clickbait',
 'Not Clickbait',
 'Not Clickbait',
 'Not Clickbait']